<a href="https://colab.research.google.com/github/JustKidding2K/ADS-CQ18-Project/blob/main/Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ĐỒ ÁN MÔN HỌC "KHOA HỌC DỮ LIỆU ỨNG DỤNG"

Giáo viên hướng dẫn: Trần Trung Kiên

Nhóm 6:

1. 18120212 - Huỳnh Long Nam

# 1. MÔ TẢ BÀI TOÁN

## 1.1 GIỚI THIỆU BÀI TOÁN

**Cuộc thi:** [Corporación Favorita Grocery Sales Forecasting](https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting)

**Mô tả bài toán**:

- **Corporación Favorita** là một nhà bán lẻ lớn từ Ecuado, với hàng hàng trăm cửa hàng và hơn 200,000 mặt hàng được kinh doanh. Hệ thống phải đối mặt với vấn đề cung ứng số lượng hàng hóa cho các cửa hàng với đặc điểm khác nhau,... Nếu nhập quá nhiều, hàng hóa sẽ bị ùn ứ, hư hỏng. Nếu nhập quá ít, cửa hàng sẽ mất khách, giảm doanh thu, lợi nhuận.
- Do đó, Corporación Favorita tổ chức cuộc thi ***Corporación Favorita Grocery Sales Forecasting*** nhằm tìm ra mô hình dự doán chính xác nhất về doanh số của các mặt hàng, giải quyết bài toán cung ứng trên.
- Các đội thi sẽ phải dự đoán doanh số bán của hàng nghìn sản phẩm tại các cửa hàng của Favorita dựa trên các thông tin về thời gian, cửa hàng, sản phẩm, sản phẩm có được quảng bá hay không và số lượng sản phẩm đã bán được(tập train)

**Số đội tham dự:** 1,671

**Giải thưởng:** $tổng 30,000 trong đó:

- $15, 000 cho đội về 1

- $10, 000 cho đội về 2

- $ 5, 000 cho đội về 3






## 1.2 SƠ LƯỢC VỀ DỮ LIỆU

Dữ liều gồm 2 file chính: 

<code>train.csv</code> 
- 125,497,041 dòng (~4.99GB)
- Các thông tin bán hàng từ 01-01-2013 đến 15-08-2017

<code>test.csv</code>
- 3,370,464 dòng (~0.12 GB)
- Các thông tin bán hàng từ 16-08-2017 đến 31-08-2017 (bỏ cột unit_sales)

Mỗi dòng bao gồm thông tin về một sản phẩm được bán ra: 

Input:
- <code>date</code>: ngày bán 
- <code>store_nbr</code>: mã cửa hàng
- <code>item_nbr</code>: mã sản phẩm
- <code>onpromotion</code>: có đang được quảng bá

Output:
- <code>unit_sales</code>: số đơn vị bán được (số lượng, khối lượng, thể tích,...)

và các file dữ liệu khác bao gồm:

<code>stores.csv</code> : thông tin về cửa hàng (store_nbr, city, state, type, cluster)

<code>items.csv</code> :  thông tin về sản phẩm (item_nbr, family, class, perishable)

<code>transactions.csv</code> : doanh số bán từng ngày của từng cửa hàng (date, store_nbr, transactions)

<code>oil.csv</code> : giá dầu hàng ngày (date, dcoilwtico)

<code>holidays_events.csv</code> : các ngày lễ tại Ecuador (date, type,locale,locale_name, description, transferred)

## 1.3 GIỚI THIỆU SOLUTION

- Solution được nhóm lựa chọn đạt được [top 1](https://www.kaggle.com/code/shixw125/1st-place-lgb-model-public-0-506-private-0-511) trên [Private Leaderboard](https://www.kaggle.com/competitions/favorita-grocery-sales-forecasting/leaderboard) với scorce <code>0.50918</code> theo độ lỗi <code>Normalized Weighted Root Mean Squared Logarithmic Error</code> (càng thấp càng tốt):

$$ NWRMSLE = \sqrt{ \frac{\sum{i=1}^n wi \left( \ln(\hat{y}i + 1) - \ln(yi +1)  \right)^2  }{\sum{i=1}^n wi}} $$

- Solution được đề cử bởi ba tác giả [pengyan](https://www.kaggle.com/pengyan), [shixw125](https://www.kaggle.com/shixw125), [infzero](https://www.kaggle.com/infzero) và được tác giả [shixw125](https://www.kaggle.com/shixw125) upload sau cuộc thi

- Solution chỉ sử dụng 59,038,132 (~ một nửa tập train)

# 2. GIẢI QUYẾT BÀI TOÁN

# 3. NHÌN LẠI QUÁ TRÌNH LÀM ĐỒ ÁN

# 4. TÀI LIỆU THAM KHẢO

EDA : [Shopping for Insights - Favorita EDA](https://www.kaggle.com/code/headsortails/shopping-for-insights-favorita-eda)

Solution: [1st Place LGB Model(public:0.506, private:0.511)](https://www.kaggle.com/code/shixw125/1st-place-lgb-model-public-0-506-private-0-511/script)